<a href="https://colab.research.google.com/github/SarahBarbosa/AluraDataChallenge_2/blob/main/S02_Feature_Engineering_e_Modelo_de_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://imgur.com/QnT31vZ.png)

---

# Sobre o desafio

Na semana 02, vamos focar na construção e otimização de modelos de machine learning para lidar com a Taxa de Churn. Agora que terminamos a limpeza e análise inicial dos dados, é hora de usar algoritmos de aprendizado de máquina para transformar nossos insights em ações eficazes.

<div style="background-color: #171821; color: white; padding: 10px; text-align: center; font-size: 20px;">
    <h1>Feature Engineearing</h1>
</div>